In [45]:
import pandas as pd
from sqlalchemy import create_engine
import yfinance as yf
from fredapi import Fred
fred_key = '9a9c69745bc32b9678c0207caa1a119a'
from datetime import datetime

In [2]:
engine = create_engine('sqlite:///Database.db')

In [3]:
start = '1990-01-01'

# FRED

In [4]:
fred_ids = ['WALCL','WTREGEN', 'RRPONTSYD', 'SP500']

In [5]:
def fred_downloader(id, start):
    from fredapi import Fred
    fred_key = '9a9c69745bc32b9678c0207caa1a119a'
    fred = Fred(api_key=fred_key)
    df = fred.get_series(series_id=id, observation_start=start)
    return df

In [6]:
#fred_downloader('WALCL', '1990-01-01')

In [10]:
def sql_importer_fred(symbol, start):
    try:
        max_date = pd.read_sql(f'SELECT MAX(index) FROM {symbol}', engine).values[0][0]
        print(max_date)
        new_data = fred_downloader(symbol, start=pd.to_datetime(max_date))
        #new_data = yf.download(symbol, start=pd.to_datetime(max_date))
        new_rows = new_data[new_data.index > max_date]
        new_rows.to_sql(symbol, engine, if_exists='append')
        print(str(len(new_rows)) + ' new rows imported to DB')
    except:
        new_data = fred_downloader(symbol, start=start)
        new_data.to_sql(symbol, engine)
        print(f'New table created for {symbol} with {str(len(new_data))} rows')

In [8]:
fred_downloader('SP500', start=start)

2013-01-07    1461.89
2013-01-08    1457.15
2013-01-09    1461.02
2013-01-10    1472.12
2013-01-11    1472.05
               ...   
2022-12-30    3839.50
2023-01-02        NaN
2023-01-03    3824.14
2023-01-04    3852.97
2023-01-05    3808.10
Length: 2609, dtype: float64

In [15]:
for id in fred_ids:
    sql_importer_fred(id, start)
    print(id + ' imported')

ValueError: Table 'WALCL' already exists.

# YFinance

In [50]:
yf_ids = ['^VIX', '^VIX3M','^GSPC']

In [53]:
def sql_importer_yf(symbol, start):
    try:
        max_date = pd.read_sql('WALCL', engine).set_index('index').index[-1]
        #max_date = pd.read_sql(f'SELECT MAX(Date) FROM {symbol}', engine).values[0][0]
        print(max_date)
        new_data = yf.download(symbol, start=pd.to_datetime(max_date))#['Adj Close'].replace('Adj Close', symbol)
        new_rows = new_data[new_data.index > max_date]
        new_rows.to_sql(symbol, engine, if_exists='append')
        print(str(len(new_rows)) + ' new rows imported to DB')
    except:
        new_data = yf.download(symbol, start=start)
        new_data.to_sql(symbol, engine)
        print(f'New table created for {symbol} with {str(len(new_data))} rows')

In [54]:
for id in yf_ids:
    sql_importer_yf(id, start)
    print(id + ' imported')

[*********************100%***********************]  1 of 1 completed
New table created for ^VIX with 8318 rows
^VIX imported
[*********************100%***********************]  1 of 1 completed
New table created for ^VIX3M with 4147 rows
^VIX3M imported
[*********************100%***********************]  1 of 1 completed
New table created for ^GSPC with 8317 rows
^GSPC imported


In [68]:
all_results = []
for id in yf_ids:
    results = pd.read_sql(id, engine).set_index('Date')
    results = results['Adj Close']
    all_results.append(results)
df = pd.concat(all_results, axis=1)
df = df.fillna(method='bfill')
df = df.fillna(method='ffill')
df = df.dropna()
df.columns = yf_ids

In [1]:
df

NameError: name 'df' is not defined

In [54]:
str(pd.read_sql('^VIX', engine).set_index('Date').index[-1].date())

'2023-01-05'

In [52]:
pd.read_sql('^VIX', engine).set_index('Date').index[-1]

Timestamp('2023-01-05 00:00:00')

In [41]:
str(pd.read_sql('WALCL', engine).set_index('index').index[-1].date())

'2023-01-04'

In [51]:
str(datetime.today().date()-1)

TypeError: unsupported operand type(s) for -: 'datetime.date' and 'int'